<a href="https://colab.research.google.com/github/rogeriomoreirajr-jornal/dataPoxa/blob/master/EmpresariosSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quem são os empresários dos jogadores dos grandes clubes catarinenses?
- Pegar os códigos dos clubes, e montar um dicionário
- Montar uma tabela com: (nome, empresario, equipe, posição, data de nascimento, na equipe desde, **outros dados**)


In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
from time import sleep
import unicodedata
from IPython.display import display, Markdown

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.lower().decode('utf8')

pd.set_option('display.max_colwidth', -1)

s = requests.Session()

def clip(df, **kwargs):
	return df.to_clipboard(float_format=CommaFloatFormatter(), sep='\t', **kwargs)

class CommaFloatFormatter:
	def __mod__(self, x):
		return str(x).replace('.',',')

def make_soup(url):
    html = s.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36'})
    soup = BeautifulSoup(html.content, "html5lib")
    return soup

def get_data(soup, string):
    try:
        str_ = soup.find('th', text=string).next_sibling.next_sibling.text
        str_ = re.search('\n? *(.+[^ ]) *\t*', str_).group(1)
        return str_.replace('\xa0\xa0','')

    except:
        return None

def get_jogador(link):
    soup_ = make_soup(base+link)

    id2_ = link.split('/')[-1]

    nome = get_data(soup_, 'Nome completo:')
    posi = get_data(soup_, 'Posição:')
    data = get_data(soup_, 'Data de nascimento:')
    dataEquipe = get_data(soup_, 'Na equipa desde:')
    empresario = get_data(soup_, 'Empresário:')

    try:
      outros_ = soup_.find('span','headerueberschrift').parent.parent
      outros_ = outros_.find('div','content').text
      outros =  re.search('[ \n\t]+(.+[^ \n])[ \n\t]+', outros_).group(1)

    except: outros = None
      
    return({'nome': nome,
            'posi': posi,
            'data': data, 
            'dataEquipe': dataEquipe, 
            'empresario': empresario, 
            'outros': outros})


base = 'https://www.transfermarkt.pt'

In [0]:
times = {'Avaí':2035,
        'Figueirense':4064,
        'Chapecoense':17776,
        'Criciúma':7178,
        'Joinville':3330}

urlTime = 'https://www.transfermarkt.pt/-/startseite/verein/{}'.format

l_ = []

for time in times:
  print(time)
  soupTime = make_soup(urlTime(times[time]))
  lJogadores = {el['href'] for el in soupTime.find('table','items')\
                .findAll('a', 'spielprofil_tooltip')}
  
  for jogador in lJogadores:
    print(jogador)
    l_.append(get_jogador(jogador))

In [37]:
df = pd.DataFrame(l_)

s = df.empresario.value_counts()

for empresario in s[s>1].index:
  display(Markdown('\n'*3+'### '+empresario))
  display(df[df.empresario==empresario])




### Marcio Bittencourt Sports

,data,dataEquipe,empresario,nome,outros,posi
48,13/04/1994,21/01/2019,Marcio Bittencourt Sports,None,None,Ataque - Segundo Avançado
50,31/01/1997,01/01/2017,Marcio Bittencourt Sports,Matheus Sousa Pereira,None,Médio - Médio Defensivo
60,29/05/1986,08/04/2019,Marcio Bittencourt Sports,Antônio de Moura Carvalho,None,Médio - Médio Ofensivo
93,23/05/1992,29/04/2019,Marcio Bittencourt Sports,None,None,Defesa - Lateral Direito
117,06/01/1993,01/01/2019,Marcio Bittencourt Sports,Isnairo Reis Silva Morais,None,Médio - Médio Ofensivo
118,22/05/1989,18/04/2019,Marcio Bittencourt Sports,Nataliel Costa da Silva,None,Médio - Médio Defensivo
139,17/05/1988,01/01/2018,Marcio Bittencourt Sports,Sandro Silva de Souza,None,Defesa - Defesa Central





### sem agente

,data,dataEquipe,empresario,nome,outros,posi
33,11/11/1983,16/08/2016,sem agente,Ebert William Amâncio,None,Defesa - Defesa Central
75,14/07/1986,12/12/2018,sem agente,Rafael Berger,None,Defesa - Defesa Central
81,18/11/1984,30/07/2018,sem agente,None,None,Defesa - Defesa Central
92,16/08/1985,12/02/2015,sem agente,Hélio Hermito Zampier Neto,None,Defesa - Defesa Central
94,09/03/1986,29/05/2019,sem agente,Fernando Camilo Farias,None,Médio - Médio Ofensivo
104,19/01/1990,01/01/2019,sem agente,None,None,Médio - Médio Direito
143,30/01/1986,22/03/2019,sem agente,Leonardo Gamalho de Souza,Grêmio Porto Alegre,Ataque - Ponta de Lança





### AGN Football

...

,data,dataEquipe,empresario,nome,outros,posi
16,10/06/1997,03/01/2019,Amin Sakman,Getulio Wandelly Silva Timóteo,None,Ataque - Ponta de Lança
28,21/04/2000,01/06/2017,Amin Sakman,Luan Pereira,None,Médio - Médio Centro





### Showball

,data,dataEquipe,empresario,nome,outros,posi
121,20/09/1999,-,Showball,Wagner Aparecido de Andrade Júnior,None,Guarda-redes
141,05/03/1987,01/01/2019,Showball,None,None,Guarda-redes





### GMRsports

,data,dataEquipe,empresario,nome,outros,posi
83,11/06/1984,05/01/2018,GMRsports,None,Mogi Mirim,Médio - Médio Defensivo
108,23/08/1989,01/01/2018,GMRsports,None,None,Médio - Médio Esquerdo
